# [WK] Projekt 1 - Opis obrazu
#### Autorzy: Agnieszka Klimek (145302), Anna Prałat (145395), Daniel Zdancewicz (145317)

## Opis zadania
Śledzenie gry w bilard.
Dla filmów przedstawiających grę w bilard (dowolna odmiana) zaproponuj system:
- Wykrywania bili
- Śledzenia bili
- Wykrywania zdarzeń na filmie
Należy samodzielnie przygotować odpowiednie dane wejściowe.

## Założenia
Jako przyjętą reprezentację obrazu przyjęto filmy gry w snookera z perspektywy ptaka (bird view).
Założono, że kamera jest nieruchoma, a stół do gry jest prostokątem. DOdatkowo założono, że
kamera jest umieszczona nad środkiem stołu, a kąt nachylenia kamery jest równy 90 stopni.

Jako zdarzenia, które są wykrywane na filmie:
- Wbicie bili do łuzy
- Zderzenie bili z drugą bilą


## Technika wykrywania bil na obrazie

1. Wczytanie obrazu
2. Przekształcenie obrazu do skali szarości
3. Zastosowanie filtru Gaussa
4. Zastosowanie binaryzacji
5. Zastosowanie operacji poprawnego wypełnienia dziur
6. Rozpoznanie 6 masek (po jednej dla każdego rodzaju bili)
7. Rozpoznanie konturów bili
8. Zapis pozycji zawierających kontury w kolejnych klatkach filmu
9. Zastosowanie prostych równań fizycznych na podstawie pozycji do wyznaczenia pozycji bili w kolejnych klatkach filmu
10. Wykrywanie zdarzeń na podstawie momentum kuli


## Wczytanie filmu

In [1]:
import cv2

match = cv2.VideoCapture('resources/matches/snooker.mp4')
